In [1]:
# This is for the cart-pole game, it's all about my experiments with the game

In [1]:
import time
import random
import heapq as hp
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000


class DQNAgent:
    def __init__(self, state_size, action_size,epsilon_decay,gamma):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = {}
        self.pqt=[]
        self.gamma = gamma   # discount rate 0.95
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = epsilon_decay  #0.99
        self.learning_rate = 0.001  # the learning rate
        self.model = self._build_model()
        self.pev_model=self.model        
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        cort_rwd=(reward + self.gamma *
                          max(self.pev_model.predict(next_state)[0]))
        prd_rwd=self.model.predict(next_state)[0][action]
        hp.heappush(self.pqt,-abs(float(cort_rwd - prd_rwd)))
        self.memory[-abs(float(cort_rwd - prd_rwd))]=(state, action, reward, next_state, done)
        
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        
        for _ in range(batch_size):
            
            state, action, reward, next_state, done = self.memory[hp.heappop(self.pqt)]
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.pev_model.predict(next_state)[0]))  # q(s,a) = r + max(q(_s,_a)) over _a
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# After 500 time steps the game automaticly terminates



def do_it(agent):
    env.reset()
    time_start=time.time()
    
    
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32
    vally=[]
    for e in range(EPISODES):

        done=False
        state = env.reset()  # It's obdervation 'o'
        state = np.reshape(state, [1, state_size]) # Encapsulating whole thing into and array i.e [[1,12,2,3]]
        for t in range(500):      # The 1st comment, actually 'for' loop will work too
            
            env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.pev_model=agent.model  # I think this should be episidic not for every t
            if done:
               # print("episode: {}/{}, score: {}, e: {:.2}"
                #      .format(e, EPISODES, t, agent.epsilon))
                break
        if t>=499 : 
            vally.append(e-1)
            #print("Time for which pole stand:",t,vally)
            if len(vally)==10:
                break
                
        if len(agent.pqt) > batch_size:
            agent.replay(batch_size)
    end_time=time.time()
    
    
    avg=0
    for i in range(100):
        #env.seed(i)
        state=env.reset()
        state=np.reshape(state,[1,agent.state_size])
        action=np.argmax(agent.model.predict(state)[0])
        nxt_state, reward, done, info = env.step(action)
        net_r=0
        while not done:
            net_r+=reward
            state=nxt_state
            state=np.reshape(state, [1,agent.state_size])
            action=np.argmax(agent.model.predict(state)[0])
            nxt_state, reward, done, info = env.step(action)
            #print(nxt_state, reward, done)
        avg+=net_r
        #print("Reward for {} episode: {}".format(i,net_r))
    
    agent.memory.clear()
    agent.pqt.clear()
    
    return([end_time-time_start, vally, avg/(i+1), agent])
        # if e % 10 == 0:
#     agent.save("./save/cartpole-dqn.h5")

In [3]:
#Set some hyperparameters gamma, epsilon_decay,min_decay
gammaLst=[0.8,0.85,0.9,0.95,1]
epsilon_decayLst=[0.5,0.7,0.8,0.9,0.95,0.98,0.985,0.99,0.995]

In [4]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent_hyp=[]
for i in range(len(gammaLst)):
    agent_gamma=[]
    for j in range(len(epsilon_decayLst)):
        finl=[]
        for po in range(3):
            finl.append(DQNAgent(state_size, action_size,epsilon_decayLst[j],gammaLst[i]))
        agent_gamma.append(finl)
    agent_hyp.append(agent_gamma)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
agent_hyp_perf=[]
crt=0
for i in range(len(gammaLst)):
    print('Gamma : ',gammaLst[i] )
    agent_gamma_perf=[]
    for j in range(len(epsilon_decayLst)):
        print('     Epsilon_Decay : ',epsilon_decayLst[j] )
        finl_perf=[]
        for po in range(3):
            finl_perf.append(do_it(agent_hyp[i][j][po]))
            print('            Trial : ',po+1 )
            print('            Time : ',finl_perf[po][0] )
            print('            Vally : ',finl_perf[po][1] )
            print('            Avg_evaluation : ',finl_perf[po][2] )
            crt+=1
            print('Done for :'+str(crt)+'/135')
        agent_gamma_perf.append([[epsilon_decayLst[j],gammaLst[i]],finl_perf])
    agent_hyp_perf.append(agent_gamma_perf)
    print("Done for gamma : ",gammaLst[i])
    print(agent_gamma_perf)

Gamma :  0.8
     Epsilon_Decay :  0.5
            Trial :  1
            Time :  128.6433756351471
            Vally :  [811, 821, 863, 883, 884, 894, 945, 950, 958]
            Avg_evaluation :  106.6
Done for :1/135
            Trial :  2
            Time :  326.5103509426117
            Vally :  [249, 254, 255, 257, 466, 528, 610, 616, 657, 678]
            Avg_evaluation :  345.63
Done for :2/135
            Trial :  3
            Time :  91.73240685462952
            Vally :  []
            Avg_evaluation :  57.56
Done for :3/135
     Epsilon_Decay :  0.7
            Trial :  1
            Time :  183.167156457901
            Vally :  [287, 299, 316, 321, 329, 331, 344, 367, 368, 371]
            Avg_evaluation :  465.11
Done for :4/135
            Trial :  2
            Time :  80.87374973297119
            Vally :  [970, 974, 980, 997, 998]
            Avg_evaluation :  36.23
Done for :5/135
            Trial :  3
            Time :  389.20196175575256
            Vally :  [897

            Trial :  1
            Time :  250.06890296936035
            Vally :  [631, 927, 945, 951, 954, 989, 991]
            Avg_evaluation :  268.71
Done for :28/135
            Trial :  2
            Time :  459.9810411930084
            Vally :  [500, 656, 666, 668, 672, 735, 766, 920, 949, 958]
            Avg_evaluation :  372.11
Done for :29/135
            Trial :  3
            Time :  193.34430623054504
            Vally :  [139, 250, 251, 271, 276, 278, 291, 293, 297, 314]
            Avg_evaluation :  499.0
Done for :30/135
     Epsilon_Decay :  0.7
            Trial :  1
            Time :  74.41242456436157
            Vally :  []
            Avg_evaluation :  8.25
Done for :31/135
            Trial :  2
            Time :  113.38109278678894
            Vally :  []
            Avg_evaluation :  110.68
Done for :32/135
            Trial :  3
            Time :  188.9111397266388
            Vally :  [210, 231, 233, 259, 262, 284, 301, 311, 314, 316]
            Avg_e

            Trial :  2
            Time :  243.91317319869995
            Vally :  [381, 506, 526, 529, 534, 542, 543, 544, 545, 546]
            Avg_evaluation :  499.0
Done for :56/135
            Trial :  3
            Time :  320.56977701187134
            Vally :  [991]
            Avg_evaluation :  165.68
Done for :57/135
     Epsilon_Decay :  0.7
            Trial :  1
            Time :  105.06221652030945
            Vally :  []
            Avg_evaluation :  59.1
Done for :58/135
            Trial :  2
            Time :  129.2839982509613
            Vally :  []
            Avg_evaluation :  31.5
Done for :59/135
            Trial :  3
            Time :  161.93837809562683
            Vally :  [863, 932, 991]
            Avg_evaluation :  118.34
Done for :60/135
     Epsilon_Decay :  0.8
            Trial :  1
            Time :  199.6062090396881
            Vally :  [273, 275, 276, 326, 346, 359, 364, 382, 401, 402]
            Avg_evaluation :  451.02
Done for :61/135
   

            Trial :  2
            Time :  221.46675658226013
            Vally :  [161, 253, 277, 279, 285, 287, 298, 303, 304, 305]
            Avg_evaluation :  471.37
Done for :83/135
            Trial :  3
            Time :  272.87622833251953
            Vally :  []
            Avg_evaluation :  207.78
Done for :84/135
     Epsilon_Decay :  0.7
            Trial :  1
            Time :  353.33474135398865
            Vally :  [296, 523, 526, 528, 531, 532, 533, 534, 535, 536]
            Avg_evaluation :  499.0
Done for :85/135
            Trial :  2
            Time :  408.8506178855896
            Vally :  [711, 712, 964, 966]
            Avg_evaluation :  208.7
Done for :86/135
            Trial :  3
            Time :  186.06990098953247
            Vally :  [467, 469, 470, 471, 473, 480, 487, 488, 489, 490]
            Avg_evaluation :  499.0
Done for :87/135
     Epsilon_Decay :  0.8
            Trial :  1
            Time :  302.639976978302
            Vally :  [174, 408

            Trial :  1
            Time :  125.77303004264832
            Vally :  []
            Avg_evaluation :  9.0
Done for :109/135
            Trial :  2
            Time :  155.89753985404968
            Vally :  []
            Avg_evaluation :  8.41
Done for :110/135
            Trial :  3
            Time :  206.16239643096924
            Vally :  []
            Avg_evaluation :  67.81
Done for :111/135
     Epsilon_Decay :  0.7
            Trial :  1
            Time :  349.4233729839325
            Vally :  []
            Avg_evaluation :  33.15
Done for :112/135
            Trial :  2
            Time :  254.98570561408997
            Vally :  [863, 882, 897, 906, 917, 927, 928, 952, 964, 965]
            Avg_evaluation :  484.94
Done for :113/135
            Trial :  3
            Time :  163.28078246116638
            Vally :  []
            Avg_evaluation :  8.43
Done for :114/135
     Epsilon_Decay :  0.8
            Trial :  1
            Time :  944.7496337890625
   

In [6]:
import os
data_root='CartPole/Phase-2/Models'
for i in range(len(gammaLst)):
    for j in range(len(epsilon_decayLst)):
        for po in range(3):
            agent_hyp[i][j][po].save(os.path.join(data_root,'G'+str(gammaLst[i])+'ED'+str(epsilon_decayLst[j])+'trail'+str(po)+'.h5'))